In [1]:
import pandas as pd
from haversine import haversine, Unit
import time
import numpy as np
import warnings

warnings.simplefilter("ignore")

In [2]:
# Load the data
properties = pd.read_csv('../data/Airport Distance/bq-unique-property-postcodes.csv')

# Load the station dataset
green = pd.read_csv('../data/environment/parks_lsoa_pre_filter.csv')


In [3]:
properties

,Postcode,Lat,Long
0,YO13 0QJ,54.303171,-0.440901
1,WA11 8NJ,53.497064,-2.773570
2,SK11 7AU,53.254723,-2.118836
3,OX17 3LW,52.014137,-1.315112
4,HP22 5XR,51.756900,-0.809104
...,...,...,...
1192273,OX12 9QL,51.588470,-1.529370
1192274,TN30 6PB,51.082571,0.694649
1192275,NG24 3DH,53.056606,-0.788587
1192276,NE15 8XA,54.983013,-1.716224


In [4]:
def clean_postcode(postcode):
    # Remove any existing spaces
    postcode = postcode.replace(' ', '')
    
    # Check if the last three characters contain a digit and two letters
    last_three = postcode[-3:]
    if last_three[0].isdigit() and last_three[1].isalpha() and last_three[2].isalpha():
        # Add a space before the last three characters
        postcode = postcode[:-3] + ' ' + postcode[-3:]
        
    return postcode

# Apply the function to the 'Postcode' column
green['Postcode'] = green['Postcode'].apply(clean_postcode)


green

,Postcode,Average_distance_to_nearest_park_or_public_garden__m_,Average_number_of_parks_or_public_gardens_within_1_000_m_radius
0,AL1 1AA,164.803376,8.324675
1,AL1 1AB,240.619939,6.076923
2,AL1 1AD,240.619939,6.076923
3,AL1 1AE,240.619939,6.076923
4,AL1 1AF,164.803376,8.324675
...,...,...,...
2193139,YO91 1WZ,1580.337968,1.000000
2193140,YO91 1XY,1580.337968,1.000000
2193141,YO95 1BT,5416.218519,1.000000
2193142,E15 1JT,254.465307,4.438662


In [5]:
# Merge the datasets - drop long/lat to simplify calc - merge properties with greenspace data
merged = pd.merge(properties, green, left_on='Postcode', right_on='Postcode', how='left')
merged.drop(columns=["Lat", "Long"], inplace=True)
merged.dropna(subset=['Average_distance_to_nearest_park_or_public_garden__m_', 'Average_number_of_parks_or_public_gardens_within_1_000_m_radius'], inplace=True)
merged

,Postcode,Average_distance_to_nearest_park_or_public_garden__m_,Average_number_of_parks_or_public_gardens_within_1_000_m_radius
0,YO13 0QJ,2056.114146,1.000000
1,WA11 8NJ,249.622984,1.000000
2,SK11 7AU,288.861546,2.743590
3,OX17 3LW,628.781389,1.000000
4,HP22 5XR,1644.970078,1.000000
...,...,...,...
1192273,OX12 9QL,2014.168450,1.000000
1192274,TN30 6PB,2327.917332,1.000000
1192275,NG24 3DH,1542.424023,1.000000
1192276,NE15 8XA,587.599386,3.186047


In [6]:
# Specify the output file path for the merged CSV file
output_file_path = f'../data/environment/parks_lsoa_filtered.csv'

# Write the merged_data DataFrame to a CSV file
merged.to_csv(output_file_path, index=False)

In [7]:
merged[merged["Postcode"] == "SW2 3BQ"]

,Postcode,Average_distance_to_nearest_park_or_public_garden__m_,Average_number_of_parks_or_public_gardens_within_1_000_m_radius
226934,SW2 3BQ,188.03204,3.724138


In [8]:
import pandas_gbq

# Use the credentials of your service account, if necessary
# pandas_gbq.context.credentials = google.auth.credentials.Credentials.from_service_account_info()

# Assuming `df` is the DataFrame you want to upload
merged.to_gbq('Green_Space.Green_Spaces_LSOA_filtered', project_id='bold-circuit-389014', if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 5949.37it/s]


In [10]:
postcodes = pd.read_csv("../data/NSPL21_MAY_2023_UK.csv")[["pcd", "pcd2", "pcds"]]
postcodes

,pcd,pcd2,pcds
0,AB1 0AA,AB1 0AA,AB1 0AA
1,AB1 0AB,AB1 0AB,AB1 0AB
2,AB1 0AD,AB1 0AD,AB1 0AD
3,AB1 0AE,AB1 0AE,AB1 0AE
4,AB1 0AF,AB1 0AF,AB1 0AF
...,...,...,...
2690846,ZE3 9JW,ZE3 9JW,ZE3 9JW
2690847,ZE3 9JX,ZE3 9JX,ZE3 9JX
2690848,ZE3 9JY,ZE3 9JY,ZE3 9JY
2690849,ZE3 9JZ,ZE3 9JZ,ZE3 9JZ


In [15]:
unique_properties_postcodes = postcodes['pcds'].unique() 
merged_postcodes = merged["Postcode"].unique()

In [17]:
common_postcodes = set(unique_properties_postcodes).intersection(set(merged_postcodes))
print(f"Number of common postcodes: {len(common_postcodes)}")

Number of common postcodes: 538912


In [18]:
# Remove spaces from the postcodes in both lists
unique_properties_postcodes_no_space = [postcode.replace(' ', '') for postcode in unique_properties_postcodes]
merged_postcodes_no_space = [postcode.replace(' ', '') for postcode in merged_postcodes]

# Calculate the intersection of the two lists
common_postcodes_no_space = set(unique_properties_postcodes_no_space).intersection(set(merged_postcodes_no_space))

print(f"Number of common postcodes without spaces: {len(common_postcodes_no_space)}")


Number of common postcodes without spaces: 1176317
